In [5]:
%run ../../../ingestion/ingestion.ipynb
%run ../../../data\ quality/quality.ipynb
%run ../../../write/write.ipynb
%run ../../../alerts/webhook.ipynb

25/05/07 21:42:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/05/07 21:42:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
# Cria uma instância da classe WebhookTeams utilizando o arquivo de configuração localizado em ../../config/teams.json
webhook = WebhookTeams("../../../config/teams.json")

In [7]:
try:
    # Cria um leitor de CSV com delimitador de linha definido como ";"
    reader = CSVReader(line_delimiter=";")
    
    # Executa a leitura do arquivo CSV e retorna um DataFrame
    df = client_code(reader, "../../../layer/bronze/saude/snis_abastecimento_de_agua.csv")
except:
    # Em caso de erro durante a leitura, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de leitura!")


In [8]:
df.show()

+---------+--------------------+--------------------+--------------------+----+--------------------+--------------------+-----+-----+------+-----+
|cod_ibge7|           municipio|               porte|                  ra| ano|             prest_a|            natjur_a|in023|in049| in051|in053|
+---------+--------------------+--------------------+--------------------+----+--------------------+--------------------+-----+-----+------+-----+
|  3500105|          Adamantina|Mais de 20 mil a ...|RA Presidente Pru...|2018|Companhia de Sane...|Sociedade de econ...|  100|23,77|132,27| 12,7|
|  3500204|              Adolfo|           At� 5 mil|RA S�o Jos� do Ri...|2018|Companhia de Sane...|Sociedade de econ...|  100| 10,8| 47,76|11,86|
|  3500402|      �guas da Prata|Mais de 5 mil a 1...|         RA Campinas|2018|Companhia de Sane...|Sociedade de econ...|  100|37,66|230,07|11,24|
|  3500501|    �guas de Lind�ia|Mais de 10 mil a ...|         RA Campinas|2018|Saneamento Ambien...|           Autarqu

In [9]:
try:
    # Cria uma instância da classe responsável por exportar em formato Parquet
    exporter = ParquetExport()

    # Exporta o DataFrame para o caminho local especificado
    exporter.export(df, "../../../layer/prata/saude/snis_abastecimento_de_agua/", destination="local")
except:
    # Em caso de erro durante a exportação, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de escrita!")
